# Using the LifeMonitor API

This notebook shows how to interact with the [LifeMonitor API](https://crs4.github.io/life_monitor/lm_api_specs). We're going to query the [LifeMonitor dev instance](https://api.dev.lifemonitor.eu) for information about the testing status of a workflow.

In [1]:
import requests

In the cell below, replace the value of `lm_api_key` with your LifeMonitor API Key. To get one, do the following:

* [Log in to LifeMonitor](https://api.dev.lifemonitor.eu/login)
* In your profile page, click on `API Keys > NEW`

In [2]:
lm_base_url = "https://api.dev.lifemonitor.eu"
lm_api_key = "PKh9hECeT_qA2RpmZ5JxDyRAxUMyitU3JnDvhN-yVybaK-gBl_zAXB"

Set up a session and populate its headers with the API Key.

In [3]:
s = requests.session()
s.headers.update({'ApiKey': lm_api_key})

Now we're ready to explore the API. Let's start by listing the available workflow registries.

In [4]:
response = s.get(f"{lm_base_url}/registries")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registries = response.json()
registries

{'items': [{'name': 'wfhubdev',
   'type': 'seek',
   'uri': 'https://dev.workflowhub.eu',
   'uuid': 'e4a937fc-276d-4954-8e98-2d0d89a3a040'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://api.dev.lifemonitor.eu',
  'resource': '/registries'}}

Let's pick the first registry of type `seek` (WorkflowHub).

In [5]:
registry_uuid = [_ for _ in registries["items"] if _["type"] == "seek"][0]["uuid"]
registry_uuid

'e4a937fc-276d-4954-8e98-2d0d89a3a040'

Now that we have the registry's UUID, we can get a list of all workflows coming from that registry that have been submitted to LifeMonitor.

In [6]:
response = s.get(f"{lm_base_url}/registries/{registry_uuid}/workflows", params={"status": False})
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
workflows = response.json()
workflows

{'items': [{'latest_version': '1',
   'name': 'COVID-19: variation analysis on ARTIC PE data',
   'uuid': '143cc7a0-8e3a-0139-2e05-005056ab5db4'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://api.dev.lifemonitor.eu',
  'resource': '/registries/e4a937fc-276d-4954-8e98-2d0d89a3a040/workflows?status=False'}}

Let's pick the first workflow from the list.

In [7]:
workflow_uuid = [_ for _ in workflows["items"]][0]["uuid"]
workflow_uuid

'143cc7a0-8e3a-0139-2e05-005056ab5db4'

Get more details about the chosen workflow.

In [8]:
response = s.get(f"{lm_base_url}/workflows/{workflow_uuid}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
workflow = response.json()
workflow

{'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://api.dev.lifemonitor.eu',
  'created': '2021-05-03T14:23:19.792342',
  'modified': '2021-05-03T14:23:19.792356',
  'resource': '/workflows/143cc7a0-8e3a-0139-2e05-005056ab5db4'},
 'name': 'COVID-19: variation analysis on ARTIC PE data',
 'registry': {'name': 'wfhubdev',
  'type': 'seek',
  'uri': 'https://dev.workflowhub.eu',
  'uuid': 'e4a937fc-276d-4954-8e98-2d0d89a3a040'},
 'uuid': '143cc7a0-8e3a-0139-2e05-005056ab5db4',
 'version': {'is_latest': True,
  'ro_crate': {'links': {'download': 'https://api.dev.lifemonitor.eu/ro_crates/11/download',
    'external': 'https://dev.workflowhub.eu/workflows/157/content_blobs/247/download'}},
  'submitter': {'id': 3, 'username': 'SimoneLeo'},
  'uuid': '3ee8b52b-7692-4b16-b91e-11a868e0d2fd',
  'version': '1'}}

Get information on the workflow's overall testing status.

In [9]:
response = s.get(f"{lm_base_url}/workflows/{workflow_uuid}/status")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
status = response.json()
for b in status['latest_builds']:
    del b['last_logs']
status

{'aggregate_test_status': 'all_passing',
 'latest_builds': [{'build_id': '769307075',
   'instance': {'managed': False,
    'name': 'test1_1',
    'resource': 'github/crs4/iwc',
    'roc_instance': '#test1_1',
    'service': {'type': 'travis',
     'url': 'https://travis-ci.org',
     'uuid': '0694c600-ffa9-4e37-bcd5-43a5790f415b'},
    'uuid': '045d99ee-1337-49bc-9465-31918bb655c3'},
   'status': 'passed',
   'suite_uuid': 'fbe0c0ae-c204-4543-ad30-b5b7d2efa875',
   'timestamp': '1620037573.0'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://api.dev.lifemonitor.eu',
  'created': '2021-05-03T14:23:19.792342',
  'modified': '2021-05-03T14:23:19.792356',
  'resource': '/workflows/143cc7a0-8e3a-0139-2e05-005056ab5db4/status'},
 'name': 'COVID-19: variation analysis on ARTIC PE data',
 'registry': {'name': 'wfhubdev',
  'type': 'seek',
  'uri': 'https://dev.workflowhub.eu',
  'uuid': 'e4a937fc-276d-4954-8e98-2d0d89a3a040'},
 'uuid': '143cc7a0-8e3a-0139-2e05-005056ab5db4',
 

List test suites for the workflow.

In [10]:
response = s.get(f"{lm_base_url}/workflows/{workflow_uuid}/suites")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
suites = response.json()
suites

{'items': [{'definition': {'test_engine': {'type': 'planemo',
     'version': '>=0.74.3'}},
   'instances': [{'managed': False,
     'name': 'test1_1',
     'resource': 'github/crs4/iwc',
     'roc_instance': '#test1_1',
     'service': {'type': 'travis',
      'url': 'https://travis-ci.org',
      'uuid': '0694c600-ffa9-4e37-bcd5-43a5790f415b'},
     'uuid': '045d99ee-1337-49bc-9465-31918bb655c3'}],
   'roc_suite': '#test1',
   'uuid': 'fbe0c0ae-c204-4543-ad30-b5b7d2efa875'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://api.dev.lifemonitor.eu',
  'resource': '/workflows/143cc7a0-8e3a-0139-2e05-005056ab5db4/suites'}}

Pick the first test suite and get its details.

In [11]:
suite_uuid = suites['items'][0]['uuid']
response = s.get(f"{lm_base_url}/suites/{suite_uuid}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
suite = response.json()
suite

{'definition': {'test_engine': {'type': 'planemo', 'version': '>=0.74.3'}},
 'instances': [{'managed': False,
   'name': 'test1_1',
   'resource': 'github/crs4/iwc',
   'roc_instance': '#test1_1',
   'service': {'type': 'travis',
    'url': 'https://travis-ci.org',
    'uuid': '0694c600-ffa9-4e37-bcd5-43a5790f415b'},
   'uuid': '045d99ee-1337-49bc-9465-31918bb655c3'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://api.dev.lifemonitor.eu',
  'resource': '/suites/fbe0c0ae-c204-4543-ad30-b5b7d2efa875'},
 'roc_suite': '#test1',
 'uuid': 'fbe0c0ae-c204-4543-ad30-b5b7d2efa875'}

Get the suite's overall test status.

In [12]:
response = s.get(f"{lm_base_url}/suites/{suite_uuid}/status")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
suite_status = response.json()
status

{'aggregate_test_status': 'all_passing',
 'latest_builds': [{'build_id': '769307075',
   'instance': {'managed': False,
    'name': 'test1_1',
    'resource': 'github/crs4/iwc',
    'roc_instance': '#test1_1',
    'service': {'type': 'travis',
     'url': 'https://travis-ci.org',
     'uuid': '0694c600-ffa9-4e37-bcd5-43a5790f415b'},
    'uuid': '045d99ee-1337-49bc-9465-31918bb655c3'},
   'status': 'passed',
   'suite_uuid': 'fbe0c0ae-c204-4543-ad30-b5b7d2efa875',
   'timestamp': '1620037573.0'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://api.dev.lifemonitor.eu',
  'created': '2021-05-03T14:23:19.792342',
  'modified': '2021-05-03T14:23:19.792356',
  'resource': '/workflows/143cc7a0-8e3a-0139-2e05-005056ab5db4/status'},
 'name': 'COVID-19: variation analysis on ARTIC PE data',
 'registry': {'name': 'wfhubdev',
  'type': 'seek',
  'uri': 'https://dev.workflowhub.eu',
  'uuid': 'e4a937fc-276d-4954-8e98-2d0d89a3a040'},
 'uuid': '143cc7a0-8e3a-0139-2e05-005056ab5db4',
 

List test instances for the chosen suite.

In [13]:
response = s.get(f"{lm_base_url}/suites/{suite_uuid}/instances")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
instances = response.json()
instances

{'items': [{'managed': False,
   'name': 'test1_1',
   'resource': 'github/crs4/iwc',
   'roc_instance': '#test1_1',
   'service': {'type': 'travis',
    'url': 'https://travis-ci.org',
    'uuid': '0694c600-ffa9-4e37-bcd5-43a5790f415b'},
   'uuid': '045d99ee-1337-49bc-9465-31918bb655c3'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://api.dev.lifemonitor.eu',
  'resource': '/suites/fbe0c0ae-c204-4543-ad30-b5b7d2efa875/instances'}}

Pick an instance and get the details.

In [14]:
instance_uuid = instances['items'][0]['uuid']
response = s.get(f"{lm_base_url}/instances/{instance_uuid}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
instance = response.json()
instance

{'managed': False,
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://api.dev.lifemonitor.eu',
  'resource': '/instances/045d99ee-1337-49bc-9465-31918bb655c3'},
 'name': 'test1_1',
 'resource': 'github/crs4/iwc',
 'roc_instance': '#test1_1',
 'service': {'type': 'travis',
  'url': 'https://travis-ci.org',
  'uuid': '0694c600-ffa9-4e37-bcd5-43a5790f415b'},
 'uuid': '045d99ee-1337-49bc-9465-31918bb655c3'}

List the latest builds for the chosen instance.

In [15]:
response = s.get(f"{lm_base_url}/instances/{instance_uuid}/latest-builds")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
latest_builds = response.json()
for b in latest_builds['items']:
    del b['last_logs']
latest_builds['items'][:3]

[{'build_id': '769307075',
  'instance': {'managed': False,
   'name': 'test1_1',
   'resource': 'github/crs4/iwc',
   'roc_instance': '#test1_1',
   'service': {'type': 'travis',
    'url': 'https://travis-ci.org',
    'uuid': '0694c600-ffa9-4e37-bcd5-43a5790f415b'},
   'uuid': '045d99ee-1337-49bc-9465-31918bb655c3'},
  'status': 'passed',
  'suite_uuid': 'fbe0c0ae-c204-4543-ad30-b5b7d2efa875',
  'timestamp': '1620037573.0'},
 {'build_id': '769294684',
  'instance': {'managed': False,
   'name': 'test1_1',
   'resource': 'github/crs4/iwc',
   'roc_instance': '#test1_1',
   'service': {'type': 'travis',
    'url': 'https://travis-ci.org',
    'uuid': '0694c600-ffa9-4e37-bcd5-43a5790f415b'},
   'uuid': '045d99ee-1337-49bc-9465-31918bb655c3'},
  'status': 'passed',
  'suite_uuid': 'fbe0c0ae-c204-4543-ad30-b5b7d2efa875',
  'timestamp': '1620027654.0'},
 {'build_id': '763952272',
  'instance': {'managed': False,
   'name': 'test1_1',
   'resource': 'github/crs4/iwc',
   'roc_instance': '#t

Pick a build and get the details.

In [16]:
build_id = latest_builds['items'][0]['build_id']
response = s.get(f"{lm_base_url}/instances/{instance_uuid}/builds/{build_id}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
build = response.json()
del build['last_logs']
build

{'build_id': '769307075',
 'instance': {'managed': False,
  'name': 'test1_1',
  'resource': 'github/crs4/iwc',
  'roc_instance': '#test1_1',
  'service': {'type': 'travis',
   'url': 'https://travis-ci.org',
   'uuid': '0694c600-ffa9-4e37-bcd5-43a5790f415b'},
  'uuid': '045d99ee-1337-49bc-9465-31918bb655c3'},
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://api.dev.lifemonitor.eu',
  'resource': '/instances/045d99ee-1337-49bc-9465-31918bb655c3/builds/769307075'},
 'status': 'passed',
 'suite_uuid': 'fbe0c0ae-c204-4543-ad30-b5b7d2efa875',
 'timestamp': '1620037573.0'}

Get the build log.

In [17]:
response = s.get(f"{lm_base_url}/instances/{instance_uuid}/builds/{build_id}/logs", params={"limit_bytes": 4*2**20})
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
log = response.json()
print(f"{log[:1300]}\n\n[...]\n\n{log[-1300:]}")

Worker information
hostname: 8a971683-252c-4d44-94ac-0ae13b161e1b@1.worker-org-dc4bd4986-dfpnv.gce-production-4
version: 6.2.22 https://github.com/travis-ci/worker/tree/858cb91994a513269f2fe9782c15fc113e966231
instance: travis-job-a86accaf-74f4-445f-a38c-027fabd57017 travis-ci-sardonyx-xenial-1593004276-4d46c6b3 (via amqp)
startup: 6.237302883s
Build system information
Build language: python
Build dist: xenial
Build id: 769307075
Job id: 769307076
Runtime kernel version: 4.15.0-1077-gcp
travis-build version: 091d532a
Build image provisioning date and time
Wed Jun 24 13:36:52 UTC 2020
Operating System Details
Distributor ID:	Ubuntu
Description:	U

[...]

tN:MainThread] Sending reconfigure_watcher control task.
galaxy.jobs.handler INFO 2021-05-03 10:52:48,290 [pN:main,p:4063,tN:MainThread] sending stop signal to worker thread
galaxy.jobs.handler INFO 2021-05-03 10:52:48,291 [pN:main,p:4063,tN:MainThread] job handler queue stopped
galaxy.jobs.runners INFO 2021-05-03 10:52:48,291 [pN:main,

For further information on the LifeMonitor API, take a look at the [docs](https://api.dev.lifemonitor.eu/openapi.html).